# 📘 LLM Summarizer Agent with Extra Features
This notebook demonstrates a more advanced summarizer agent using LLMs.

Features:
- Multiple summary modes (concise, bullets, insights, TL;DR)
- Tone control (neutral, professional, casual, academic, persuasive)
- Target length (word-based)
- Summarize from text, URL, or PDF

In [ ]:
!pip install openai requests PyPDF2 beautifulsoup4

In [ ]:

import requests
from bs4 import BeautifulSoup
import PyPDF2
from openai import OpenAI

class LLMSummarizerAgent:

    def __init__(self, model="gpt-4o-mini"):

        self.model = model

        self.client = OpenAI()


    def summarize(self, text: str, mode: str = "concise", target_words: int = 120, tone: str = "neutral") -> str:

        """

        Summarize text in different styles, lengths, and tones.


        :param text: Input text.

        :param mode: "concise", "bullets", "insights", "tldr".

        :param target_words: Approximate word target.

        :param tone: Writing tone ("neutral", "professional", "casual", "academic", "persuasive").

        """

        instructions = {

            "concise": "Write one concise paragraph.",

            "bullets": "Summarize in 5-8 bullet points.",

            "insights": "Write 3-5 key insights (executive style).",

            "tldr": "Write a single TL;DR line."

        }

        prompt = f"""

        Summarize the following text in about {target_words} words.

        Mode: {mode}.

        Tone: {tone}.

        Instructions: {instructions.get(mode, 'Write a concise summary.')}


        Text:

        {text}

        """


        resp = self.client.chat.completions.create(

            model=self.model,

            messages=[{"role": "user", "content": prompt}],

            temperature=0.4

        )

        return resp.choices[0].message.content.strip()


    def summarize_url(self, url: str, **kwargs) -> str:

        """Fetch webpage text and summarize."""

        html = requests.get(url).text

        soup = BeautifulSoup(html, "html.parser")

        text = " ".join(p.get_text() for p in soup.find_all("p"))

        return self.summarize(text, **kwargs)


    def summarize_pdf(self, file_path: str, **kwargs) -> str:

        """Extract text from PDF and summarize."""

        with open(file_path, "rb") as f:

            reader = PyPDF2.PdfReader(f)

            text = " ".join(page.extract_text() or "" for page in reader.pages)

        return self.summarize(text, **kwargs)



In [ ]:

agent = LLMSummarizerAgent()



long_text = """

Artificial intelligence (AI) refers to computer systems capable of performing tasks

that typically require human intelligence. These tasks include learning, reasoning,

problem-solving, and language understanding.

"""



# Neutral tone, concise

print("Concise Neutral:", agent.summarize(long_text, mode="concise", tone="neutral"))



# Professional tone, bullet points

print("Bullets Professional:", agent.summarize(long_text, mode="bullets", tone="professional"))



# Casual tone, TL;DR

print("Casual TL;DR:", agent.summarize(long_text, mode="tldr", tone="casual"))



In [ ]:

# Summarize from a URL (academic tone)

# print(agent.summarize_url("https://openai.com/research", mode="insights", tone="academic"))


# Summarize from a PDF file (TL;DR)

# print(agent.summarize_pdf("sample.pdf", mode="tldr", tone="neutral"))

